In [1]:
import pandas as pd
import numpy  as np

import pickle 

from pathlib import Path

import time


import sys
sys.path.append("D:/analyze_Pearson_residuals")

from  nru_DE import *


pd.set_option('display.max_columns', 5)

In [2]:
np.random.seed( 12345 ) 

In [3]:
#### user specified
data_folder = r"D:/analyze_Pearson_residuals/"

data_subfolder = "33k_PBMC"

data_path = Path ( data_folder + data_subfolder )

In [4]:
# output data
results_pkl = "Ag_complementary_samples.pkl"
gene_array_pkl  = "gene_array_complementary_samples.pkl"  # for scanpy and SCTransform
df_cells_pkl  = "df_cells_complementary_samples.pkl"      # for scanpy and SCTransform
 
# input data
nru_dict_pkl =  "nru_dict.pkl"
counts_pkl = "counts_sparse.pkl" # sparse pandas data frame: cells in columns; genes in rows
 


# paths: output data
results_dsn = data_path / results_pkl
gene_array_dsn = data_path / gene_array_pkl
df_cells_dsn = data_path / df_cells_pkl

# paths: input data
nru_dict_dsn = data_path / nru_dict_pkl
counts_dsn = data_path / counts_pkl

In [5]:
pctl_list = [.01,.05, .10, .25, .5, .75, .90, .95, .99 ]

In [6]:
nz_min_complemetary = 50
n_samples = 2

In [7]:
f = open( nru_dict_dsn, 'rb' )    
nru_dict = pickle.load(f)           
f.close()       

df_selected_cells = nru_dict['df_selected_cells']
print ( ' df_selected_cells:  ' )
print ( '\n', df_selected_cells )

df_gene_stats = nru_dict['df_gene_stats'] [[ 'nz_cells', 'M_g', 'A_g']]
print ( '\n\n df_gene_stats:  ' )
print ( '\n', df_gene_stats )

 df_selected_cells:  

                      0      1   ...     38     39
AAACATTGACGACT-1  False   True  ...  False   True
AAACATTGACGGGA-1   True  False  ...   True  False
AAACATTGCCGCTT-1   True  False  ...   True  False
AAACATTGCTCATT-1   True  False  ...   True  False
AAACCGTGCCCTCA-1  False   True  ...   True  False
...                 ...    ...  ...    ...    ...
TTTGCATGCACTAG-6   True  False  ...  False   True
TTTGCATGCCGAAT-6  False   True  ...   True  False
TTTGCATGGAGGTG-6  False   True  ...   True  False
TTTGCATGGATAAG-6  False   True  ...   True  False
TTTGCATGGGCATT-6   True  False  ...   True  False

[33148 rows x 40 columns]


 df_gene_stats:  

                  nz_cells         M_g         A_g
ENSG00000254709      1679  594.080034  321.476696
ENSG00000163736       679   85.392337   78.464313
ENSG00000087086     32296   43.064119   42.226038
ENSG00000163220     10159   36.166104   34.965024
ENSG00000090382     15500   34.801999   34.258216
...                   ...  

In [8]:
df_counts_sparse = pd.read_pickle ( counts_dsn )
print ( ' df_counts_sparse:' )
print ( df_counts_sparse )
print ( '\n type( df_counts_sparse ) ', type ( df_counts_sparse ))
print ( '\n df_counts_sparse.sparse.density: ', df_counts_sparse.sparse.density)

 df_counts_sparse:
                 AAACATTGACGACT-1  AAACATTGACGGGA-1  ...  TTTGCATGGATAAG-6  \
ENSEMBL_ID                                           ...                     
ENSG00000239945                 0                 0  ...                 0   
ENSG00000237683                 0                 0  ...                 0   
ENSG00000228463                 0                 0  ...                 0   
ENSG00000235373                 0                 0  ...                 0   
ENSG00000228327                 0                 0  ...                 0   
...                           ...               ...  ...               ...   
ENSG00000212884                 0                 0  ...                 0   
ENSG00000215615                 0                 0  ...                 0   
ENSG00000215764                 0                 0  ...                 0   
ENSG00000215700                 1                 0  ...                 1   
ENSG00000215699                 0            

In [9]:
df_counts_sparse_gene_sel = df_counts_sparse.loc [ df_gene_stats.index ]
print ( ' df_counts_sparse_gene_sel:' )
print ( df_counts_sparse_gene_sel )
print ( '\n type( df_counts_sparse_gene_sel ) ', type ( df_counts_sparse_gene_sel ))
print ( '\n df_counts_sparse_gene_sel.sparse.density: ', df_counts_sparse_gene_sel.sparse.density)

del df_counts_sparse

 df_counts_sparse_gene_sel:
                 AAACATTGACGACT-1  AAACATTGACGGGA-1  ...  TTTGCATGGATAAG-6  \
ENSG00000254709                 0                 0  ...                 0   
ENSG00000163736                 0                 0  ...                 0   
ENSG00000087086                 3                 6  ...                 8   
ENSG00000163220                 0                 0  ...                 0   
ENSG00000090382                 1                 0  ...                 0   
...                           ...               ...  ...               ...   
ENSG00000171241                 0                 0  ...                 0   
ENSG00000149573                 0                 0  ...                 0   
ENSG00000122025                 0                 0  ...                 0   
ENSG00000237980                 0                 0  ...                 0   
ENSG00000158488                 0                 0  ...                 0   

                 TTTGCATGGGCATT-6  

In [10]:
df_sample_nz_totals_list = []
df_sample_nz_totals = pd.DataFrame ( index=df_counts_sparse_gene_sel.index )


print ( 'for each gene, compute number of nonzero cells in each UMI count sub-matrix' )
for sample in  range(n_samples):
  print ( '\n sample: ', sample )
  
  df_cell_sample = df_selected_cells[[sample]]
  df_cell_select = df_cell_sample [ df_cell_sample[sample] ]
  barcode_list = df_cell_select.index.values.tolist()
  
  df_counts_sparse_sample = df_counts_sparse_gene_sel[ barcode_list ]  
  df_counts_GT_0 = ( df_counts_sparse_sample > 0 ).astype( int )  
  df_sample_nz_sum = df_counts_GT_0.sum ( axis=1 ).to_frame ( name = sample )
  df_sample_nz_totals_list.append ( df_sample_nz_sum )
  
df_sample_nz_totals = pd.concat ( df_sample_nz_totals_list , axis=1 )  
df_sample_nz_totals['min_nz'] = df_sample_nz_totals.min ( axis=1 )

print ( '\n df_sample_nz_totals' )
print ( df_sample_nz_totals  )
print ( '\n\n df_sample_nz_totals.describe' )
print ( df_sample_nz_totals.describe ( percentiles=pctl_list ) )

for each gene, compute number of nonzero cells in each UMI count sub-matrix

 sample:  0

 sample:  1

 df_sample_nz_totals
                     0      1  min_nz
ENSG00000254709    836    843     836
ENSG00000163736    317    362     317
ENSG00000087086  16162  16134   16134
ENSG00000163220   5113   5046    5046
ENSG00000090382   7715   7785    7715
...                ...    ...     ...
ENSG00000171241     34     22      22
ENSG00000149573     27     36      27
ENSG00000122025     59     65      59
ENSG00000237980     29     23      23
ENSG00000158488     29     44      29

[12324 rows x 3 columns]


 df_sample_nz_totals.describe
                  0             1        min_nz
count  12324.000000  12324.000000  12324.000000
mean    1024.917803   1027.313129   1013.704722
std     2118.982546   2119.036391   2113.669066
min       16.000000     15.000000     15.000000
1%        26.000000     25.000000     24.000000
5%        35.000000     35.000000     32.000000
10%       48.000000     49

In [11]:
gene_select_array = df_sample_nz_totals[ df_sample_nz_totals['min_nz'] >= nz_min_complemetary ].index.values

df_counts_sparse_all_samples = df_counts_sparse_gene_sel.loc [ gene_select_array ]
print ( '\n df_counts_sparse_all_samples' )
print ( df_counts_sparse_all_samples  )

del df_counts_sparse_gene_sel


 df_counts_sparse_all_samples
                 AAACATTGACGACT-1  AAACATTGACGGGA-1  ...  TTTGCATGGATAAG-6  \
ENSG00000254709                 0                 0  ...                 0   
ENSG00000163736                 0                 0  ...                 0   
ENSG00000087086                 3                 6  ...                 8   
ENSG00000163220                 0                 0  ...                 0   
ENSG00000090382                 1                 0  ...                 0   
...                           ...               ...  ...               ...   
ENSG00000060982                 0                 0  ...                 0   
ENSG00000156113                 0                 0  ...                 0   
ENSG00000135549                 0                 0  ...                 0   
ENSG00000155962                 0                 0  ...                 0   
ENSG00000122025                 0                 0  ...                 0   

                 TTTGCATGGGCATT-

In [12]:
#### verify that all cells have nonzero sequencing depth
ser_cell_totals = df_counts_sparse_all_samples.sum ( axis=0 ) 
print ( '\n\n ser_cell_totals.describe' )
print ( ser_cell_totals.describe ( percentiles=pctl_list ) )

ser_cell_totals_GT_0 = ser_cell_totals[ ser_cell_totals > 0 ]
print ( '\n\n ser_cell_totals_GT_0.describe' )
print ( ser_cell_totals_GT_0.describe ( percentiles=pctl_list ) )



 ser_cell_totals.describe
count    33148.000000
mean      2051.592012
std        965.069756
min        476.000000
1%         569.000000
5%         815.000000
10%       1059.000000
25%       1458.000000
50%       1886.000000
75%       2431.000000
90%       3207.300000
95%       3827.000000
99%       5282.890000
max      15456.000000
dtype: float64


 ser_cell_totals_GT_0.describe
count    33148.000000
mean      2051.592012
std        965.069756
min        476.000000
1%         569.000000
5%         815.000000
10%       1059.000000
25%       1458.000000
50%       1886.000000
75%       2431.000000
90%       3207.300000
95%       3827.000000
99%       5282.890000
max      15456.000000
dtype: float64


In [13]:
df_counts_sparse_all_samples_SD_positive  = df_counts_sparse_all_samples[ ser_cell_totals_GT_0.index ]
print ( ' df_counts_sparse_all_samples_SD_positive:' )
print ( df_counts_sparse_all_samples_SD_positive )
print ( '\n type( df_counts_sparse_all_samples_SD_positive ) ', type ( df_counts_sparse_all_samples_SD_positive ))
print ( '\n df_counts_sparse_all_samples_SD_positive.sparse.density: ', df_counts_sparse_all_samples_SD_positive.sparse.density)

 df_counts_sparse_all_samples_SD_positive:
                 AAACATTGACGACT-1  AAACATTGACGGGA-1  ...  TTTGCATGGATAAG-6  \
ENSG00000254709                 0                 0  ...                 0   
ENSG00000163736                 0                 0  ...                 0   
ENSG00000087086                 3                 6  ...                 8   
ENSG00000163220                 0                 0  ...                 0   
ENSG00000090382                 1                 0  ...                 0   
...                           ...               ...  ...               ...   
ENSG00000060982                 0                 0  ...                 0   
ENSG00000156113                 0                 0  ...                 0   
ENSG00000135549                 0                 0  ...                 0   
ENSG00000155962                 0                 0  ...                 0   
ENSG00000122025                 0                 0  ...                 0   

                 TTT

In [14]:
df_cells_SD_positive = pd.DataFrame ( index = df_counts_sparse_all_samples_SD_positive.columns, data=True, columns=['SD_positive'] )


df_complementary_selected_cells_list = []

for sample in  range(n_samples):
  print ( '\n sample: ', sample )
  df_cell_sample = pd.concat ( [ df_selected_cells[[sample]], df_cells_SD_positive ], axis=1 )
  df_cell_sample = df_cell_sample.assign( SD_positive = df_cell_sample['SD_positive'].fillna(False) )
  df_complementary_selected_cells_list.append ( df_cell_sample.all ( axis=1 ).to_frame ( name = sample )   ) 

df_complementary_selected_cells = pd.concat ( df_complementary_selected_cells_list, axis=1 )  
  
print ( '\n df_complementary_selected_cells' )
print ( df_complementary_selected_cells  )
print ( '\n\n df_complementary_selected_cells.sum' )
print ( df_complementary_selected_cells.sum () )


 sample:  0

 sample:  1

 df_complementary_selected_cells
                      0      1
AAACATTGACGACT-1  False   True
AAACATTGACGGGA-1   True  False
AAACATTGCCGCTT-1   True  False
AAACATTGCTCATT-1   True  False
AAACCGTGCCCTCA-1  False   True
...                 ...    ...
TTTGCATGCACTAG-6   True  False
TTTGCATGCCGAAT-6  False   True
TTTGCATGGAGGTG-6  False   True
TTTGCATGGATAAG-6  False   True
TTTGCATGGGCATT-6   True  False

[33148 rows x 2 columns]


 df_complementary_selected_cells.sum
0    16593
1    16555
dtype: int64


In [15]:
print ( 'calculations for samples' )

df_gene_stats_half_cell_samples_list = []
  
for sample in  range(n_samples):
  print ( '\n sample: ', sample )
  df_cell_sample = df_complementary_selected_cells[[sample]]
  df_cell_select = df_cell_sample [ df_cell_sample[sample] ]
  barcode_list = df_cell_select.index.values.tolist() 
  df_counts_sparse_sample_analy = df_counts_sparse_all_samples_SD_positive[ barcode_list ]  
 
  start_time = time.time()
  nru_dict = nru ( df_counts_sparse_sample_analy , nz_min=nz_min_complemetary , n_genes=5 ) 
  end_time = time.time()
  elapsed = end_time - start_time
  print ( 'function nru elapsed time: ',  f"{ elapsed:.1f}", ' seconds'  )

  df_gene_stats_half_cell_sample = nru_dict['df_gene_stats'] [['A_g']].rename ( columns={'A_g': sample} )
  df_gene_stats_half_cell_sample.sort_values ( [sample], ascending=False, inplace=True )
 
  df_gene_stats_half_cell_samples_list.append ( df_gene_stats_half_cell_sample )
  
  
df_gene_stats_half_cell_samples = pd.concat ( df_gene_stats_half_cell_samples_list, axis=1 )
print ( '\n df_gene_stats_half_cell_samples' )
print ( df_gene_stats_half_cell_samples  )
print ( '\n\n df_gene_stats_half_cell_samples.describe' )
print ( df_gene_stats_half_cell_samples.describe ( percentiles=pctl_list ) )

calculations for samples

 sample:  0
counts_sparse_selected_genes.shape:  (10908, 16593)
counts_sparse_selected_csr.shape:  (10908, 16593)
calculating sum of squares of Pearson residuals using all cells
calculating sum of squares of Pearson residuals using cell sample  0
calculating sum of squares of Pearson residuals using cell sample  1
calculating sum of squares of Pearson residuals using cell sample  2
calculating sum of squares of Pearson residuals using cell sample  3
calculating sum of squares of Pearson residuals using cell sample  4
calculating sum of squares of Pearson residuals using cell sample  5
calculating sum of squares of Pearson residuals using cell sample  6
calculating sum of squares of Pearson residuals using cell sample  7
calculating sum of squares of Pearson residuals using cell sample  8
calculating sum of squares of Pearson residuals using cell sample  9
calculating sum of squares of Pearson residuals using cell sample  10
calculating sum of squares of Pearso

In [16]:
df_gene_stats_half_cell_samples.to_pickle ( results_dsn )
df_complementary_selected_cells.to_pickle ( df_cells_dsn )

f = open( gene_array_dsn, 'wb' )    
pickle.dump ( gene_select_array, f )
f.close()  